In [100]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder

In [101]:
df = pd.read_csv(r"C:\Users\Romar\Google Drive\Master\DE\DE-group-5\Assignment 2\Data\data.csv")
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [102]:
df.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingT

In [103]:
def PreprocessData(df):
    #Remove all NaN rows
    df_preprocessed = df.dropna()
    
    #one hot encoding nationality
    encoder = TargetEncoder()
    df['Nationality Encoded'] = encoder.fit_transform(df['Nationality'], df['Overall'])


    #one hot encoding club (remove NaN first)
    encoder1 = TargetEncoder()
    df['Club Encoded'] = encoder1.fit_transform(df['Club'], df['Overall'])
    #onehot encoding preferred foot (remove NaN first)
    #encode Workrate
    #encode BodyType
    #Pencode osostion
    #Value
    #Wage
    wage = df_preprocessed.Wage
    newwage = []
    for item in wage:
        temp = item.split("€")[1]
        newwage.append(float(temp.split("K")[0]))
    df_preprocessed['Wage'] = newwage
    newwage = []
    
    #Weight
    weight = df_preprocessed.Weight
    newweight = []
    for item  in weight:
        newweight.append(float(item.split("lbs")[0]))
    df_preprocessed['Weight'] = newweight
    newweight = []
    #Height
    newheight = []
    height =df_preprocessed.Height
    for item in height:
        feet, inch = float(item.split("'")[0]), float(item.split("'")[1])
        newheight.append(feet*30.48 + inch*2.54)
    df_preprocessed['Height'] = newheight
    newheight = []
    #All the positions, weet niet wat we daar mee willlen

    return df_preprocessed
def SplitData(df):
    df.pop('ID')
    df.pop('Unnamed: 0')
    df.pop('Photo')
    df.pop('Flag')
    df.pop('Club Logo')
    df.pop('Release Clause')
    df.pop('Name')
    df.pop('Loaned From')
    df.pop('Real Face')
    df.pop('Contract Valid Until')

    df_pp = PreprocessData(df)

    y = df_pp.pop('Overall')
    X_train, X_test, y_train, y_test = train_test_split(df_pp, y, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = SplitData(df)

<ipython-input-103-7222c60b7443>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_preprocessed['Wage'] = newwage
<ipython-input-103-7222c60b7443>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_preprocessed['Weight'] = newweight
<ipython-input-103-7222c60b7443>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [104]:
x = X_train.columns
for item in x:
    print(item)
    print(X_train[(item)].dtype)

Age
int64
Nationality
object
Potential
int64
Club
object
Value
object
Wage
float64
Special
int64
Preferred Foot
object
International Reputation
float64
Weak Foot
float64
Skill Moves
float64
Work Rate
object
Body Type
object
Position
object
Jersey Number
float64
Joined
object
Height
float64
Weight
float64
LS
object
ST
object
RS
object
LW
object
LF
object
CF
object
RF
object
RW
object
LAM
object
CAM
object
RAM
object
LM
object
LCM
object
CM
object
RCM
object
RM
object
LWB
object
LDM
object
CDM
object
RDM
object
RWB
object
LB
object
LCB
object
CB
object
RCB
object
RB
object
Crossing
float64
Finishing
float64
HeadingAccuracy
float64
ShortPassing
float64
Volleys
float64
Dribbling
float64
Curve
float64
FKAccuracy
float64
LongPassing
float64
BallControl
float64
Acceleration
float64
SprintSpeed
float64
Agility
float64
Reactions
float64
Balance
float64
ShotPower
float64
Jumping
float64
Stamina
float64
Strength
float64
LongShots
float64
Aggression
float64
Interceptions
float64
Positioning
float6

In [105]:
X_train.describe()

,Age,Potential,Wage,Special,International Reputation,Weak Foot,Skill Moves,Jersey Number,Height,Weight,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
count,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,...,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000,9883.000000
mean,25.133664,71.313973,9.863301,1666.772943,1.112618,3.001012,2.522109,19.550643,180.356191,164.099565,...,52.085804,60.311039,51.463118,52.151270,49.859354,10.520591,10.571689,10.569969,10.557422,10.528888
std,4.613809,6.076061,22.634894,198.823434,0.389559,0.630949,0.613707,15.627259,6.460291,14.793601,...,12.487594,10.267343,17.201884,19.085415,19.129731,3.091903,3.082354,3.154498,3.072855,3.093309
min,16.000000,48.000000,1.000000,1000.000000,1.000000,1.000000,2.000000,2.000000,154.940000,110.000000,...,14.000000,30.000000,10.000000,10.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,21.000000,67.000000,1.000000,1525.000000,1.000000,3.000000,2.000000,8.000000,175.260000,154.000000,...,42.000000,53.000000,38.000000,36.000000,32.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,25.000000,71.000000,3.000000,1671.000000,1.000000,3.000000,2.000000,17.000000,180.340000,163.000000,...,52.000000,61.000000,56.000000,59.000000,56.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,28.000000,75.000000,9.000000,1807.000000,1.000000,3.000000,3.000000,26.000000,185.420000,174.000000,...,61.000000,68.000000,65.000000,67.000000,65.000000,13.000000,13.000000,13.000000,13.000000,13.000000
max,41.000000,95.000000,455.000000,2346.000000,5.000000,5.000000,5.000000,99.000000,203.200000,243.000000,...,92.000000,95.000000,94.000000,92.000000,91.000000,32.000000,33.000000,41.000000,33.000000,37.000000


In [94]:
test = X_train.N[1]
test

405.0

'405'